# Accessing HSDS from the HDF5 library

<div class="alert alert-block alert-warning">
This notebook will work only with a running HSDS instance. If you don't have one running, you can start one using the <a href="./05-HSDS.ipynb">HSDS notebook</a>.
</div>

## Introducing the HDF5 REST VOL connector

### Installing the HDF5 REST VOL connector

- We need a newer version of HDF5 than is available in the Debian repositories.  We'll install from source.
- We'll also need to install the HDF5 REST VOL connector from source.

In [ ]:
%%bash
git clone https://github.com/HDFGroup/vol-rest.git build/rest-vol
mkdir -p build/rest-vol/src
git clone https://github.com/HDFGroup/hdf5.git build/rest-vol/src/hdf5
cd build/rest-vol
./build_vol_autotools.sh -P /home/vscode/.local -H /home/vscode/.local 2>&1 > /dev/null


### Modifying the C++ HDF5 example to use the REST VOL connector

## Improving performance with multi-dataset I/O

In [ ]:
%%writefile src/restvol.h
#ifndef RESTVOL_H
#define RESTVOL_H

#define HSDS_FILENAME_PREFIX "/home/vscode/"

#define FILENAME_BUFFER_SIZE 1024
#define NUM_DATASETS 2
#define NUM_INTEGERS (1000 * 1000 * 100) / NUM_DATASETS

#define ERROR(msg)                \
    fprintf(stderr, "%s\n", msg); \
    exit(1);

#define LOG(msg) \
    fprintf(stderr, "%s\n", msg);

// For displaying log messages after the progress bars for dataset read/write
#define SCROLL_AMNT (NUM_DATASETS < 21 ? NUM_DATASETS : 21)

#endif

In [ ]:
%%writefile src/restvol.c
#include "restvol.h"
#include "hdf5.h"
#include "rest_vol_public.h"

#include <stdlib.h>

int main(void)
{
    LOG("Initializing REST VOL...")
    H5rest_init();

    // Set a file access property to use the REST VOL
    hid_t fapl = H5Pcreate(H5P_FILE_ACCESS);
    H5Pset_fapl_rest_vol(fapl);

    LOG("Creating file...")
    hid_t file = H5I_INVALID_HID;
    if ((file = H5Fcreate(HSDS_FILENAME_PREFIX "my_file.h5", H5F_ACC_TRUNC, H5P_DEFAULT, fapl)) < 0)
    {
        ERROR("Failed to create file");
    }

    LOG("Setting up data...");
    fprintf(stderr, "  %d Datasets of %d integers each\n", NUM_DATASETS, NUM_INTEGERS);

    int *write_data[NUM_DATASETS];
    int *read_data[NUM_DATASETS];

    hid_t dspace_ids[NUM_DATASETS];
    hid_t dset_ids[NUM_DATASETS];
    hid_t type_ids[NUM_DATASETS];
    hid_t sel_space_ids[NUM_DATASETS];

    for (size_t i = 0; i < NUM_DATASETS; i++)
    {
        dspace_ids[i] = H5Screate_simple(1, (hsize_t[]){NUM_INTEGERS}, NULL);

        char dset_name[FILENAME_BUFFER_SIZE];
        snprintf(dset_name, FILENAME_BUFFER_SIZE, "%s%zu", "dset", i);

        if ((dset_ids[i] = H5Dcreate(file, dset_name, H5T_NATIVE_INT, dspace_ids[i],
                                     H5P_DEFAULT, H5P_DEFAULT, H5P_DEFAULT)) < 0)
        {
            ERROR("Failed to create dataset");
        }

        type_ids[i] = H5T_NATIVE_INT;
        sel_space_ids[i] = H5S_ALL;

        if ((write_data[i] = calloc(NUM_INTEGERS, sizeof(int))) == NULL)
        {
            ERROR("Failed to allocate memory");
        }

        if ((read_data[i] = calloc(NUM_INTEGERS, sizeof(int))) == NULL)
        {
            ERROR("Failed to allocate memory");
        }

        for (size_t j = 0; j < NUM_INTEGERS; j++)
            write_data[i][j] = i * 10000 + j;
    }

    LOG("Writing data...");

    if (H5Dwrite_multi(NUM_DATASETS, dset_ids, type_ids, sel_space_ids, sel_space_ids,
                       H5P_DEFAULT, (const void **)write_data) < 0)
    {
        ERROR("Failed to perform multi write");
    }

    for (size_t i = 0; i < SCROLL_AMNT; i++)
        fprintf(stderr, "\n");

    LOG("Write complete. Reading data...");
    fflush(stderr);

    if (H5Dread_multi(NUM_DATASETS, dset_ids, type_ids, sel_space_ids, sel_space_ids,
                      H5P_DEFAULT, (void **)read_data) < 0)
    {
        ERROR("Failed to perform multi read");
    }

    for (size_t i = 0; i < SCROLL_AMNT; i++)
        fprintf(stderr, "\n");

    LOG("Verifying read correctness...");

    for (size_t i = 0; i < NUM_DATASETS; i++)
        for (size_t j = 0; j < NUM_INTEGERS; j++)
        {
            if (read_data[i][j] != write_data[i][j])
            {
                fprintf(stderr, "At read_data[%zu][%zu], %d != expected %d\n",
                        i, j, read_data[i][j], write_data[i][j]);
                exit(1);
            }
        }

    LOG("Multi read/write complete!")

    H5Pclose(fapl);
    H5Fclose(file);

    LOG("Terminating REST VOL...");
    H5rest_term();

    return 0;
}

In [6]:
%%bash
gcc -I/home/vscode/.local/include -L/home/vscode/.local/lib ./src/restvol.c -o ./build/restvol -lhdf5 -lhdf5_vol_rest -lcurl -ldl -Wl,-rpath=/home/vscode/.local/lib
export HSDS_USERNAME=vscode
export HSDS_PASSWORD=vscode
export HSDS_ENDPOINT=http://localhost:5101
./build/restvol

Initializing REST VOL...


Creating file...
Setting up data...
  2 Datasets of 50000000 integers each
Writing data...


Write complete. Reading data...


Verifying read correctness...
Multi read/write complete!
Terminating REST VOL...
